## **Reranking from Scratch using different Techniques like BM25,CrossEncoder,CohereAPI**


In [9]:
documents = [
    "This is a list which containing sample documents.",
    "Keywords are important for keyword-based search.",
    "Document analysis involves extracting keywords.",
    "Keyword-based search relies on sparse embeddings.",
    "Understanding document structure aids in keyword extraction.",
    "Efficient keyword extraction enhances search accuracy.",
    "Semantic similarity improves document retrieval performance.",
    "Machine learning algorithms can optimize keyword extraction methods."
]

In [3]:
!pip install sentence-transformers

In [4]:
model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1"

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
documents

['This is a list which containing sample documents.',
 'Keywords are important for keyword-based search.',
 'Document analysis involves extracting keywords.',
 'Keyword-based search relies on sparse embeddings.',
 'Understanding document structure aids in keyword extraction.',
 'Efficient keyword extraction enhances search accuracy.',
 'Semantic similarity improves document retrieval performance.',
 'Machine learning algorithms can optimize keyword extraction methods.']

In [11]:
len(documents)

8

In [12]:
documents_embeddings=model.encode(documents)

In [13]:
len(documents_embeddings)

8

In [14]:
len(documents_embeddings[0])

768

In [24]:
for i,embedding in enumerate(documents_embeddings):
  print(f"Documents {i+1}\n{documents[i]}: \nembeddings{embedding[:5]}")

Documents 1
This is a list which containing sample documents.: 
embeddings[ 0.10894687  0.07812072  0.11626563 -0.31912503  0.46890235]
Documents 2
Keywords are important for keyword-based search.: 
embeddings[0.36544177 0.41045633 0.02045547 0.04747327 0.15103115]
Documents 3
Document analysis involves extracting keywords.: 
embeddings[0.28732774 0.15202944 0.24311854 0.11179013 0.23771577]
Documents 4
Keyword-based search relies on sparse embeddings.: 
embeddings[ 0.05179755  0.34211177  0.00411031  0.21485412 -0.01834122]
Documents 5
Understanding document structure aids in keyword extraction.: 
embeddings[ 0.33541682  0.17370166  0.13414086  0.3169742  -0.07318446]
Documents 6
Efficient keyword extraction enhances search accuracy.: 
embeddings[ 0.34392154  0.23954627  0.16326727  0.15590808 -0.2132833 ]
Documents 7
Semantic similarity improves document retrieval performance.: 
embeddings[0.16869691 0.37198654 0.2555241  0.24098764 0.20074865]
Documents 8
Machine learning algorithms

In [25]:
query = "Natural language processing techniques enhance keyword extraction efficiency."

In [26]:
query_embedding = model.encode(query)

In [27]:
print(f"Query Embeddings:{query_embedding[:5]}")

Query Embeddings:[0.34282106 0.3261703  0.00773253 0.01607591 0.02385168]


In [28]:
len(query_embedding)

768

### **Simiraity search**

In [29]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
similarities = cosine_similarity(np.array([query_embedding]),documents_embeddings)

In [32]:
similarities

array([[0.16948149, 0.4580228 , 0.5675695 , 0.441233  , 0.6316118 ,
        0.75214136, 0.550352  , 0.74481666]], dtype=float32)

In [33]:
most_similar_index=np.argmax(similarities)

In [34]:
most_similar_index

5

In [36]:
most_similar_document=documents[most_similar_index]
most_similar_document

'Efficient keyword extraction enhances search accuracy.'

In [37]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [38]:
similarities[0]

array([0.16948149, 0.4580228 , 0.5675695 , 0.441233  , 0.6316118 ,
       0.75214136, 0.550352  , 0.74481666], dtype=float32)

In [39]:
similarity_score=similarities[0][most_similar_index]

In [40]:
similarity_score

0.75214136

In [45]:
sorted_indices=np.argsort(similarities[0])[::-1]

In [46]:
sorted_indices

array([5, 7, 4, 2, 6, 1, 3, 0])

In [48]:
ranked_documents=[(documents[i],similarities[0][i]) for i in sorted_indices]
ranked_documents

[('Efficient keyword extraction enhances search accuracy.', 0.75214136),
 ('Machine learning algorithms can optimize keyword extraction methods.',
  0.74481666),
 ('Understanding document structure aids in keyword extraction.', 0.6316118),
 ('Document analysis involves extracting keywords.', 0.5675695),
 ('Semantic similarity improves document retrieval performance.', 0.550352),
 ('Keywords are important for keyword-based search.', 0.4580228),
 ('Keyword-based search relies on sparse embeddings.', 0.441233),
 ('This is a list which containing sample documents.', 0.16948149)]

In [49]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [53]:
print("Ranked Documents:")
for rank,(document,similarity) in enumerate(ranked_documents,start=1):
  print(f"Rank {rank}: Document-'{document}'Similarity Score-{similarity}")

Ranked Documents:
Rank 1: Document-'Efficient keyword extraction enhances search accuracy.'Similarity Score-0.7521413564682007
Rank 2: Document-'Machine learning algorithms can optimize keyword extraction methods.'Similarity Score-0.7448166608810425
Rank 3: Document-'Understanding document structure aids in keyword extraction.'Similarity Score-0.6316118240356445
Rank 4: Document-'Document analysis involves extracting keywords.'Similarity Score-0.5675694942474365
Rank 5: Document-'Semantic similarity improves document retrieval performance.'Similarity Score-0.5503519773483276
Rank 6: Document-'Keywords are important for keyword-based search.'Similarity Score-0.458022803068161
Rank 7: Document-'Keyword-based search relies on sparse embeddings.'Similarity Score-0.4412330090999603
Rank 8: Document-'This is a list which containing sample documents.'Similarity Score-0.16948148608207703


In [56]:
print("Top 4 Ranked Documents:")
for rank,(document,similarity) in enumerate(ranked_documents[:4],start=1):
  print(f"Rank {rank}: Document-'{document}'Similarity Score-{similarity}")

Top 4 Ranked Documents:
Rank 1: Document-'Efficient keyword extraction enhances search accuracy.'Similarity Score-0.7521413564682007
Rank 2: Document-'Machine learning algorithms can optimize keyword extraction methods.'Similarity Score-0.7448166608810425
Rank 3: Document-'Understanding document structure aids in keyword extraction.'Similarity Score-0.6316118240356445
Rank 4: Document-'Document analysis involves extracting keywords.'Similarity Score-0.5675694942474365


In [55]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

###**Reranking the documents in the ranked documents using BM25 library**

In [57]:
!pip install rank_bm25

In [58]:
from rank_bm25 import BM25Okapi

In [59]:
top_4_documents=[doc[0] for doc in ranked_documents[:4]]

In [60]:
top_4_documents

['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [61]:
tokenizer_top_4_documents=[doc.split() for doc in top_4_documents]

In [62]:
tokenizer_top_4_documents

[['Efficient', 'keyword', 'extraction', 'enhances', 'search', 'accuracy.'],
 ['Machine',
  'learning',
  'algorithms',
  'can',
  'optimize',
  'keyword',
  'extraction',
  'methods.'],
 ['Understanding',
  'document',
  'structure',
  'aids',
  'in',
  'keyword',
  'extraction.'],
 ['Document', 'analysis', 'involves', 'extracting', 'keywords.']]

In [63]:
bm25=BM25Okapi(tokenizer_top_4_documents)

In [64]:
tokenizer_query=query.split()
tokenizer_query

['Natural',
 'language',
 'processing',
 'techniques',
 'enhance',
 'keyword',
 'extraction',
 'efficiency.']

In [65]:
bm25_scores=bm25.get_scores(tokenizer_query)

In [66]:
bm25_scores

array([0.1907998 , 0.16686672, 0.17803252, 0.        ])

In [67]:
sorted_indices_2=np.argsort(bm25_scores)[::-1]
sorted_indices_2

array([0, 2, 1, 3])

In [68]:
top_4_documents

['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [69]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [70]:
reranked_documents=[(top_4_documents[i],bm25_scores[i]) for i in sorted_indices_2]
reranked_documents

[('Efficient keyword extraction enhances search accuracy.',
  0.19079979534096053),
 ('Understanding document structure aids in keyword extraction.',
  0.1780325227902643),
 ('Machine learning algorithms can optimize keyword extraction methods.',
  0.1668667199671815),
 ('Document analysis involves extracting keywords.', 0.0)]

In [71]:
print("Top 4 ReRanked Documents:")
for rank,(document,similarity) in enumerate(reranked_documents,start=1):
  print(f"Rank {rank}: Document-'{document}'Similarity Score-{similarity}")

Top 4 ReRanked Documents:
Rank 1: Document-'Efficient keyword extraction enhances search accuracy.'Similarity Score-0.19079979534096053
Rank 2: Document-'Understanding document structure aids in keyword extraction.'Similarity Score-0.1780325227902643
Rank 3: Document-'Machine learning algorithms can optimize keyword extraction methods.'Similarity Score-0.1668667199671815
Rank 4: Document-'Document analysis involves extracting keywords.'Similarity Score-0.0


###**Reranking the documents in the ranked documents using Cross Encoders library**

In [72]:
from sentence_transformers import CrossEncoder

In [73]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [74]:
top_4_documents

['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [75]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [76]:
pairs=[]
for doc in top_4_documents:
  pairs.append([query,doc])
pairs

[['Natural language processing techniques enhance keyword extraction efficiency.',
  'Efficient keyword extraction enhances search accuracy.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Machine learning algorithms can optimize keyword extraction methods.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Understanding document structure aids in keyword extraction.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Document analysis involves extracting keywords.']]

In [77]:
scores = cross_encoder.predict(pairs)
scores

array([ 3.1378708 ,  0.84216565, -2.9193    , -2.878192  ], dtype=float32)

In [78]:
scored_docs=zip(scores,top_4_documents)

In [79]:
scored_docs

In [80]:
reranked_documents_cross_encoders=sorted(scored_docs,reverse=True)

In [81]:
reranked_documents_cross_encoders

[(3.1378708, 'Efficient keyword extraction enhances search accuracy.'),
 (0.84216565,
  'Machine learning algorithms can optimize keyword extraction methods.'),
 (-2.878192, 'Document analysis involves extracting keywords.'),
 (-2.9193, 'Understanding document structure aids in keyword extraction.')]

###**Reranking the documents in the ranked documents using CohereAPI**

In [82]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00


In [83]:
top_4_documents

['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [84]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [85]:
import cohere

In [92]:
from google.colab import userdata
api_key=userdata.get('cohere_api_key')

In [94]:
co = cohere.Client(api_key)

In [96]:
response=co.rerank(
    model="rerank-english-v3.0",
    query=query,
    documents=top_4_documents,
    return_documents=True
)

In [97]:
print(response)

id='69eac2c0-506b-4949-8779-d36cafe2babf' results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Efficient keyword extraction enhances search accuracy.'), index=0, relevance_score=0.99411184), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Machine learning algorithms can optimize keyword extraction methods.'), index=1, relevance_score=0.9129032), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Understanding document structure aids in keyword extraction.'), index=2, relevance_score=0.32885265), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Document analysis involves extracting keywords.'), index=3, relevance_score=0.02865267)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [101]:
response.results[0].document.text

'Efficient keyword extraction enhances search accuracy.'

In [103]:
response.results[0].relevance_score

0.99411184

In [109]:
for i in range(4):
  print("TEXT:",response.results[i].document.text,"\n","SCORE",response.results[i].relevance_score)

TEXT: Efficient keyword extraction enhances search accuracy. 
 SCORE 0.99411184
TEXT: Machine learning algorithms can optimize keyword extraction methods. 
 SCORE 0.9129032
TEXT: Understanding document structure aids in keyword extraction. 
 SCORE 0.32885265
TEXT: Document analysis involves extracting keywords. 
 SCORE 0.02865267


In [110]:
reranked_documents_CohereAPI=[(response.results[i].document.text,response.results[i].relevance_score) for i in range(4)]
reranked_documents_CohereAPI

[('Efficient keyword extraction enhances search accuracy.', 0.99411184),
 ('Machine learning algorithms can optimize keyword extraction methods.',
  0.9129032),
 ('Understanding document structure aids in keyword extraction.', 0.32885265),
 ('Document analysis involves extracting keywords.', 0.02865267)]